In [1]:
from utilities import *

In [2]:
pathname = "recds_yg/"
filenames = []
for root, dirs, files in os.walk(pathname):
    for name in files:     
        filename = pathname + name
        filenames.append(filename)
            
hmm_digit = get_hmm_digits(filenames)

In [3]:
num_filter = 40
num_states = 5
num_digits = 7

In [4]:
## hmm_digit: 10 hmms for one digit recognized
## change transition_matrix 
for i in range(len(hmm_digit)):
    hmm_digit[i].transition_matrix[num_states-1][num_states-1] = 0.5

In [5]:
## non emitting state
class neState:
    def __init__(self,id,transition_prob=0.5,parent=None,next=None):
        self.id = id
        self.transition_prob = transition_prob
        self.parent = parent ## prev as a list of state4
        self.next = next ## next as a list of state0

In [6]:
## assign name to state
hmm_digits = []
for i in range(7):
    if i == 0:
        hmm_di = copy.deepcopy(hmm_digit[2:])
    else:
        hmm_di = copy.deepcopy(hmm_digit)
    
    for j in range(len(hmm_di)):
        hmm = hmm_di[j]
        for k in range(len(hmm.states)):
            hmm.states[k].name = -j    
    hmm_digits.append(hmm_di)

In [7]:
hmm_digits[3][4].states[2].name

-4

In [8]:
hmm_digits[6][4].states[4].name

-4

In [9]:
## nonemiting states list
nonemitingStates = []
for i in range(num_digits+1):
    nonemitingState = neState(id=i)
    #print(nonemitingState)
    nonemitingStates.append(nonemitingState)
    

In [11]:
## reconstruct and add nonemitting state
## assign .parent and .next to state
## assign transition prob to state

for i in range(len(hmm_digits)):
    nonemitingState_prev = nonemitingStates[i]
    nonemitingState_post = nonemitingStates[i+1]
    hmm_digit= hmm_digits[i]
    
    for j in range(len(hmm_digit)):
        hmm = hmm_digit[j]
        transition_matrix = hmm.transition_matrix
        for k in range(len(hmm.states)):
            if k == 0:
                ## parent: nonemitting + self
                parent1 = (nonemitingState_prev,1)
                parent2 = (hmm.states[k],transition_matrix[k][k])   
                ## child: next + self
                children1 = (hmm.states[k+1],transition_matrix[k][k+1])
                children2 = (hmm.states[k],transition_matrix[k][k])
            elif k == num_states-1:
                ## parent: prev + self
                parent1 = (hmm.states[k-1],transition_matrix[k-1][k])
                parent2 = (hmm.states[k],transition_matrix[k][k]) 
                ## children: nonemitting + self
                children1 = (nonemitingState_post,0.5)
                children2 = (hmm.states[k],transition_matrix[k][k])
            else:
                ## parent: prev + self
                parent1 = (hmm.states[k-1],transition_matrix[k-1][k])
                parent2 = (hmm.states[k],transition_matrix[k][k]) 
                ## children: next + self
                children1 = (hmm.states[k+1],transition_matrix[k][k+1])
                children2 = (hmm.states[k],transition_matrix[k][k])
            hmm.states[k].parent = [parent1,parent2]
            hmm.states[k].next = [children1,children2]


In [12]:
## add nonemiting state
nonemitingState0 = nonemitingStates[0]
child = []
for i in range(len(hmm_digits[0])):
    child.append(hmm_digits[0][i].states[0])
child.append(nonemitingStates[3])
nonemitingState0.next = child

for i in range(1,len(nonemitingStates)):
    nonemitingState = nonemitingStates[i]
    ## connect last state of previous hmm_digit and nonemitting state
    hmm_digit_prev = hmm_digits[i-1]
    parents = []
    for j in range(len(hmm_digit_prev)):
        parents.append(hmm_digit_prev[j].states[num_states-1])
    nonemitingState.parent = parents
    ## connect nonemitting state and first state of next hmm_digit
    if i != 7:
        hmm_digit_post = hmm_digits[i]
        child = [] 
        for k in range(len(hmm_digit_post)):
            child.append(hmm_digit_post[k].states[0])
        nonemitingState.next = child
        
## special case for nonemitingstate3
nonemitingState3 = nonemitingStates[3]
parents = nonemitingState3.parent
parents.append(nonemitingState0)
nonemitingState3.parent = parents

In [13]:
nonemitingStates[6].next

In [16]:
nonemitingStates[6]

In [14]:
hmm_digits[6][9].states[0].parent

[(<__main__.neState at 0x7f82ebfbe4c0>, 1),
 (<utilities.State at 0x7f82ebfe0f40>, 0.8387096774193549)]

In [17]:
hmm_digits[6][9].states[0].parent[0][0].next

In [18]:
## build tree
states = []
cnt = -1
for i in range(len(nonemitingStates)):
    cur_neState = nonemitingStates[i]
    cnt += 1
    cur_neState.id = cnt
    states.append(cur_neState)
    if i != 7:
        hmm_digit = hmm_digits[i]
        for j in range(len(hmm_digit)):
            hmm = hmm_digit[j]
            for k in range(len(hmm.states)):
                cnt += 1
                hmm.states[k].id = cnt
                states.append(hmm.states[k])

In [24]:
states[300].next[0][0].id

301

In [19]:
states[7].next

[(<utilities.State at 0x7f82ebecad90>, 0.19230769230769232),
 (<utilities.State at 0x7f82ebecaa90>, 0.8076923076923077)]

In [23]:
states[7].next[1][0].id

7

In [21]:
states.index(nonemitingStates[3])

143